### 数据加载

In [ ]:
from source.load_excel_to_table import load_excel_to_table

part_a_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "甲方", "part_a")
part_b_conn = load_excel_to_table("/Users/sqjian/Documents/codespace/financial_tools/testdata/test.xlsx", "乙方", "part_b")

### 提取分类条件

In [ ]:
import rich
from source.get_group_conditions import get_group_conditions


part_a_group_col_rst = get_group_conditions(part_a_conn, "part_a", "统一仓库", "统一名称", "统一日期")
part_b_group_col_rst = get_group_conditions(part_b_conn, "part_b", "统一仓库", "统一名称", "统一日期")

rich.print(part_a_group_col_rst)

### 分组提取匹配情况

In [ ]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data

group1_exit = False
group2_exit = False

for group1 in part_a_group_col_rst["group1_col"]:
    if group1_exit:
        break
    group1_exit = True
    for group2 in part_a_group_col_rst["group2_col"]:
        if group2_exit:
            break
        group2_exit = True
        for group3 in part_a_group_col_rst["group3_col"]:
            list_a = get_group_data(part_a_conn, "part_a", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "收货数量")
            list_b = get_group_data(part_b_conn, "part_b", "统一仓库", group1, "统一名称", group2, "统一日期", group3, "实发数量")
            rst = reconcile_lists(list_a, list_b)
            if all(len(value) == 0 for value in rst.values()):
                continue
            rich.print(f"{group1}=>{group2}=>{group3}", rst)

### 单点测试

In [ ]:
import rich
from source.reconcile_lists import reconcile_lists
from source.get_group_data import get_group_data
from collections import Counter

list_a = get_group_data(part_a_conn, "part_a", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "收货数量")
list_b = get_group_data(part_b_conn, "part_b", "统一仓库", "安徽合肥仓", "统一名称", "徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN", "统一日期", "2024-07-01 00:00:00", "实发数量")
_rst = reconcile_lists(list_a, list_b)
rich.print(_rst)

### 移除匹配的数据

匹配条件
- 实发批号的时间需要大于收货批号
- 两个时间尽可能相邻
- 数量相等

In [ ]:
from string import Template


def process_exact_matches(exact_matches):
    _part_b_sql_query_tmpl = """
SELECT
	*
FROM
	main.part_b
WHERE
	统一仓库 = '安徽合肥仓'
	AND 统一名称 = '徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN'
	AND 统一日期 = '2024-07-01 00:00:00'
	AND 实发数量 = ${exact_match_key}
limit ${exact_match_val}
"""
    _part_b_sql_delete_tmpl = """
DELETE
FROM
	main.part_b
WHERE
	id IN (
	SELECT
		id
	FROM
		main.part_b pb
	WHERE
		统一仓库 = '安徽合肥仓'
		AND 统一名称 = '徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN'
		AND 统一日期 = '2024-07-01 00:00:00'
		AND 实发数量 = ${exact_match_key}
	LIMIT ${exact_match_val}
)
RETURNING *
;
"""

    for exact_match_key, exact_match_val in exact_matches.items():
        part_b_ts = part_b_conn.sql(Template(_part_b_sql_query_tmpl.strip()).substitute(exact_match_key=exact_match_key, exact_match_val=exact_match_val)).fetchdf()
        print(part_b_ts["发货月份"].values[0] if len(part_b_ts) > 0 else part_b_ts)
        print("--------------------")
        part_b_del_stats = part_b_conn.sql(Template(_part_b_sql_delete_tmpl.strip()).substitute(exact_match_key=exact_match_key, exact_match_val=exact_match_val))
        print(part_b_del_stats)


process_exact_matches(_rst["exact_matches"])

In [ ]:
result = part_a_conn.sql(f"DELETE FROM part_a WHERE 统一仓库 = '安徽合肥仓' AND 统一名称 = '徐福记沙琪玛香酥芝麻味袋装@5(20x32g) N1 CN' AND 统一日期 = '2024-07-01 00:00:00' AND 收货数量 = 10 RETURNING *")
print(len(result))